In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import glob
from glob import glob

datasets_folder_path = 'C:/Users/Chotu/Videos/Major/'

In [2]:
def getting_all_essays(folder_path):
  #recursively find all .csv files in directory and sub-directory
  all_csv_files = [file for path, subdir, files in os.walk(folder_path)
                          for file in glob(os.path.join(path, "*.csv"))]
  #print(all_csv_files)
  return pd.concat([pd.read_csv(f) for f in all_csv_files], ignore_index = True)


In [3]:
D = getting_all_essays(datasets_folder_path)
y = D['domain1_score']
print(len(y))

600


In [4]:
P = pd.read_csv( 'C:/Users/Chotu/Videos/hindi_stopwords.csv')
stop_words=P['stopwords']
stop_words

0       अगर
1        अत
2      अथवा
3      अंदर
4       अदि
       ... 
299    होति
300    होती
301    होते
302    होना
303    होने
Name: stopwords, Length: 304, dtype: object

In [5]:
import nltk
#nltk.download('punkt')
from string import digits 
import re
import string
from nltk.tokenize import word_tokenize
sentenceEnders = re.compile('[।!?]')
ge = re.compile(' ')
wordy1 = []
wordy2= []
wordy3=[]
flag = 0
train_essay = D['essay']
for i in range(len(train_essay)):
    m=[]
    if i>=0:
        essay_v = train_essay[i]
        essay_v =re.sub("\u200b","",essay_v)
        essay_v.strip()
        #print(sentenceEnders.split(essay_v))
        
        X = sentenceEnders.split(essay_v)
        #print(X)
        wordy=[]
        for sentence in X:
            p = []
            #print(ge.split(sentence))
            #words = ge.split(sentence)
            tokens = word_tokenize(sentence)
            
            remove_digits = str.maketrans('', '', digits) 
            list = [i.translate(remove_digits) for i in tokens] 
            
            table = str.maketrans('', '', string.punctuation)
            words = [w.translate(table) for w in list ]
            #print(words)
            for word in words:
                #print(word)
                flag = 0
                for stop_word in stop_words:
                    #print(stop_word)
                    #print("word "+word)
                    #print("-----///")
                    if word == stop_word or word == '':
                        flag = 1
                        #print(word+" "+stop_word)
                if flag == 0:
                    #print(word)
                    wordy.append(word)
                    p.append(word)
            if len(p)>1:
                wordy2.append(p)
                m.append(p)
            
            """print("Before\n")
            #print(words)
            print("\n------\n")
            print("After\n")
            #print(wordy)
            
            print("\n")
            """
    
        wordy1.append(wordy)
        
        #print("-----")
    wordy3.append(m)
    #print("\n\n\n")


In [6]:
#print(wordy1)


In [7]:
print(len(train_essay))
print('wordy1')
print(len(wordy1))
print(wordy1[0])
print('\nwordy3')
print(len(wordy3))
print(wordy3[0])

600
wordy1
600
['प्रिय', 'स्थानीय', 'समाचार', 'पत्र', 'मुझे', 'लगता', 'कंप्यूटर', 'लोगों', 'पड़ने', 'प्रभाव', 'महान', 'शिक्षण', 'कौशल', 'प्रभावित', 'क्योंकि', 'हमें', 'दोस्तों', 'नए', 'लोगों', 'चैट', 'देते', 'हमें', 'ग्लोब', 'खगोल', 'विज्ञान', 'बारे', 'जानने', 'मदद', 'हमें', 'रोकते', 'बारे', 'आपको', 'लगता', 'किशोर', 'हमेशा', 'दोस्तों', 'फोन', 'रहता', 'आपको', 'कैसा', 'लगेगा', 'चीजों', 'बारे', 'दोस्तों', 'व्यपर', 'साथी', 'चैट', 'खैर', 'कंप्यूटर', 'चैट', 'नया', 'तरीका', 'इंटरनेट', 'सारी', 'साइटें', 'संगठन', 'संगठन', 'फेसबुक', 'माईस्पेस', 'ईत्यादी', 'सोचिए', 'कंप्यूटर', 'बॉस', 'मिलना', 'शुरू', 'किशोर', 'फोन', 'मस्ती', 'क्योंकि', 'इस्तेमाल', 'चाहते', 'आपने', 'देशों', 'राज्यों', 'बारे', 'सीखा', 'वैसे', 'मेरे', 'पास', 'कंप्यूटर', 'इंटरनेट', 'हमारे', 'चल', 'जानने', 'नया', 'तरीका', 'सोच', 'बच्चा', 'कंप्यूटर', 'बिताता', 'उनसे', 'अर्थव्यवस्था', 'समुद्री', 'तल', 'फैलने', 'दिनांक', 'बारे', 'इतना', 'सवाल', 'पूछें', 'जानते', 'आश्चर्यचकित', 'रह', 'जाएंगे', 'कंप्यूटर', 'दिलचस्प', 'कक्षा', 'दिन', 'किताब

In [8]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.2))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

Using TensorFlow backend.


In [9]:
import numpy as np

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    #print(len(words))
    index2word_set = set(model.wv.index2word)#contains all important the words of dataset
    for mword in words:
        if mword in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[mword]) 
    #print(num_words)
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [11]:
#import sklearn
from sklearn import model_selection
#from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cv = KFold( n_splits=2, shuffle=True, random_state=42)
print(cv)
results = []
result1=[]
y_pred = []
y_tr=[]
X=train_essay
count = 1
for traincv, testcv in cv.split(X):
    print("\n--------Fold {}--------\n".format(count))
    #print(traincv, testcv)
    
    y_test, y_train = y.iloc[testcv], y.iloc[traincv]
    
    train_essays =[]
    test_essays = []
    sentences=[]
    for i in traincv:
        train_essays.append(wordy1[i])
        sentences+=wordy3[i]
    
    for i in testcv:
        test_essays.append(wordy1[i])
    #print(len(sentences))
    
    num_features = 300 
    min_word_count = 10
    num_workers = 4
    context = 16
    downsampling = 1e-1
    
    
    print("Training Word2Vec Model...")
    model=Word2Vec(sentences,workers=num_workers,size=num_features,min_count=min_word_count,window=context,sample=downsampling,sg=1)

    #print(model)
    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)
    #print(len(model.wv.index2word))
    print(model['समाचार'])
    trainDataVecs = getAvgFeatureVecs(train_essays,model,num_features)
    testDataVecs = getAvgFeatureVecs(test_essays ,model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    #print(len(trainDataVecs))
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    print(trainDataVecs.shape)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    #print(trainDataVecs)
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=8, epochs=50)
    #lstm_model.load_weights('C:/Users/Chotu/Videos/final_lstm.h5')
    y_tr=lstm_model.predict(trainDataVecs)
    y_pred = lstm_model.predict(testDataVecs)
    
    # Save any one of the 8 models.
    #if count == 5:
     #    lstm_model.save('C:/Users/Chotu/Videos/final_lstm.h5')
    
    # Round y_pred to the nearest integer.
    y_tr = np.around(y_tr)
    y_pred = np.around(y_pred)
    #print((y_pred))
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test,y_pred,weights='quadratic')
    result1 = cohen_kappa_score(y_test,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result1))
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1

KFold(n_splits=2, random_state=42, shuffle=True)

--------Fold 1--------

Training Word2Vec Model...


C:\Users\Chotu\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:51: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\Chotu\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


[-0.04073243  0.02042876  0.12234924  0.05511399  0.0834847   0.06172904
  0.03367629  0.00716771  0.005562   -0.02836733 -0.01794094  0.04231445
  0.06338049 -0.03338175  0.05095775 -0.04568664  0.05929884 -0.05703808
 -0.03766166  0.02719336  0.05416233  0.04580848 -0.02766491  0.02713832
  0.04999499 -0.020891   -0.00244502  0.03430922  0.00171622  0.06397095
  0.10243507 -0.03782377  0.0977569   0.04352505 -0.07548579  0.08360419
  0.00867173 -0.06701899 -0.01120926  0.08486607 -0.01969915 -0.05378792
 -0.08276778  0.108992   -0.07679977  0.07893147  0.05301336  0.00932922
  0.00741024  0.0354628  -0.05071997  0.12840542  0.06956049 -0.00194527
  0.05465797 -0.01131261  0.01831352 -0.02111728  0.01201671  0.04820081
 -0.0465324   0.00083661 -0.0064483   0.03488184  0.09773426  0.08216082
  0.06889141 -0.03977772 -0.02090098  0.07121605  0.06087074 -0.00236995
  0.00995281 -0.04624975 -0.10225923 -0.01818929  0.09366198  0.00922456
 -0.03038746  0.05677593 -0.05033971 -0.08234379 -0

KeyboardInterrupt: 

In [ ]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))